# RAG(Retrieval-augmented generation)

https://docs.langchain.com/oss/python/langchain/rag


자체 문서를 사용하여 응답을 생성하는 앱 구현

**LangChain v1.2+ 기준**
최신 LangChain은 `langchain-core`, `langchain-community`, `langchain-text-splitters` 그리고 각 파트너 패키지(예: `langchain-chroma`, `langchain-openai`)로 모듈화되어 있습니다. 본 실습은 최신 표준인 **LCEL(LangChain Expression Language)** 패턴을 따릅니다.

**주요 개념**
- Document
- Vector Stores
- Retrievers


## RAG 프로세스


**Phase1: Indexing**

![](https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_indexing.png?w=840&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=1838328a870c7353c42bf1cc2290a779)

1. **로드**: 먼저 데이터를 로드해야 한다. 이를 위해 Document Loader를 사용한다.

2. **분할**: Text Splitter는 큰 문서를 작은 청크로 분할한다. 이는 데이터를 인덱싱하거나 모델에 전달할 때 유용하며, 큰 청크는 검색이 어렵고 모델의 제한된 컨텍스트 윈도우에 맞지 않기 때문이다.

3. **저장**: 분할된 청크를 저장하고 인덱싱할 장소가 필요하다. 이를 위해 보통 VectorStore와 Embeddings 모델을 사용한다.



**Phase2: Retrieval & Generation**

![](https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_retrieval_generation.png?w=840&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=67fe2302e241fc24238a5df1cf56573d)

4. **검색**: 사용자의 입력이 주어지면, Retriever를 사용하여 저장소에서 관련된 청크를 검색한다.

5. **생성**: ChatModel 또는 LLM은 질문과 검색된 데이터를 포함하는 프롬프트를 사용해 답변을 생성한다.


![](https://python.langchain.com/assets/images/rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png)



In [1]:
%pip install -Uq langchain langchain-community langchain-openai langchain-chroma pypdf

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  # 현재 경로의 .env를 읽어 환경변수로 등록
os.environ['LANGSMITH_TRACING'] = 'true'  # LangSmith 트레이싱(추적) 기능 활성화
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'  # LangSmith API 엔드포인트 설정
os.environ['LANGSMITH_API_KEY'] = os.getenv('langsmith_key')          # .env의 langsmith_key 값을 LANGSMITH_API_KEY 키로 설정
os.environ['LANGSMITH_PROJECT'] = 'skn23-langchain'                   # LangSmith 프로젝트명 설정
os.environ['OPENAI_API_KEY'] = os.getenv("openai_key")                # .env의 openai_key 값을 OPENAI_API_KEY 키로 설정

## 1.Indexing Phase
![](https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_indexing.png?w=840&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=1838328a870c7353c42bf1cc2290a779)


In [3]:
# pdf 데이터 로딩
!gdown 1DRrdZ5XroNSgIO9ydWPrLup1XDAAv9qO

Downloading...
From: https://drive.google.com/uc?id=1DRrdZ5XroNSgIO9ydWPrLup1XDAAv9qO
To: c:\Users\Playdata\llm\06_2-stage_rag\snow-white.pdf

  0%|          | 0.00/148k [00:00<?, ?B/s]
100%|██████████| 148k/148k [00:00<00:00, 1.44MB/s]
100%|██████████| 148k/148k [00:00<00:00, 1.43MB/s]


In [4]:
# Load
# PyPDFLoader : 로컬 PDF 파일을 페이지 단위 Document 객체 리스트로 변환하는 로더
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('./snow-white.pdf')  # PDF 파일 경로 지정
docs = loader.load()                      # PDF를 페이지별 Document 리스트로 로드
print(len(docs))  # 로드된 페이지 개수

for doc in docs:
    print(doc)               # Document 객체
    print(doc.metadata)      # 페이지 메타데이터(source, page 등 정보)
    print(doc.page_content)  # 텍스트 내용
    print('\n\n')

c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



6
page_content='백설공주
옛날 어느 왕국에 공주님이 태어났어요.
“어쩜 이렇게 어여쁠까? 살결이 눈처럼 하얗구나. 백
설공주라고 불러야겠다.”
왕과 왕비는 갓 태어난 딸을 보며 기뻐했어요.
하지만 기쁨도 잠시, 왕비는 곧 세상을 떠나고 말았어
요.' metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'moddate': '2023-09-12T11:20:24+09:00', 'source': './snow-white.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}
{'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'moddate': '2023-09-12T11:20:24+09:00', 'source': './snow-white.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}
백설공주
옛날 어느 왕국에 공주님이 태어났어요.
“어쩜 이렇게 어여쁠까? 살결이 눈처럼 하얗구나. 백
설공주라고 불러야겠다.”
왕과 왕비는 갓 태어난 딸을 보며 기뻐했어요.
하지만 기쁨도 잠시, 왕비는 곧 세상을 떠나고 말았어
요.



page_content='왕은 아름다운 새 왕비를 맞았어요.
그런데 새 왕비는 자기보다 아름다운 사람을 두고 보
지 못했어요.
왕비는 진실만을 말하는 요술 거울에게 늘 이렇게 물
었어요.
“거울아, 거울아. 이 세상

#### Text Splitter

**RecursiveCharacterTextSplitter**

긴 텍스트를 재귀적으로 분석하여 작은 조각으로 분할하는 TextSplitter의 구현체

- 문서를 작은 청크로 분할하는 데 사용한다.
- 재귀적 접근 - 큰 텍스트를 작은 조각으로 분할하다가, 원하는 크기로 나눌 수 없으면 더 작은 구분자를 사용하여 계속 나눈다.
- overlap 처리
    - 긴 텍스트를 조각으로 나눌 때 중요한 문맥이 조각 간에 손실되는 것을 방지.
    - 특히 텍스트 분류나 요약처럼 문맥이 중요한 작업에 필수적.
    - 문장완료 전에 청크가 나뉘어진 경우 overlap을 사용하여 문맥을 보존할 수 있다.

**매개변수**
- chunk_size 각 조각의 최대 문자 수를 정의한다. 기본값은 보통 1000.
- chunk_overlap 인접한 텍스트 조각 간 겹치는 문자 수를 정의한다. 중요한 문맥 손실을 방지하기 위해 설정한다(기본값: 200).
- separators 텍스트를 분할하기 위한 구분자의 우선순위를 설정한다.
    - 기본값: ["\n\n", "\n", " ", ""] (문단 → 줄바꿈 → 공백 → 문자 단위).

**작동 방식**
1. 텍스트를 가장 큰 구분자(예: 문단)를 기준으로 나눈다.
2. 나뉜 조각 중 크기가 chunk_size를 초과하면, 더 작은 구분자(예: 줄바꿈 또는 단어)를 사용해 나눈다.
3. 재귀적으로 작업하여 모든 조각이 chunk_size를 충족할 때까지 반복한다.

In [5]:
# RecursiveCharacterTextSplitter : 구분자를 기준으로 재귀적으로 잘라 Chunk_size에 맞추는 분할기(스플리터)
from langchain_text_splitters import RecursiveCharacterTextSplitter

text = """
## 인공지능 시스템의 도덕적 행위자성과 책임 소재에 관한 고찰

### 1. 서론

현대 사회에서 인공지능(AI)은 단순한 도구를 넘어 의사결정의 주체로 진화하고 있다. 자율주행 자동차, 의료 진단 알고리즘, 그리고 금융 자동 거래 시스템 등은 인간의 직접적인 개입 없이도 중대한 결과를 초래한다. 이러한 기술적 진보는 우리에게 중요한 철학적 질문을 던진다. 인공지능이 내린 결정으로 인해 피해가 발생했을 때, 그 책임은 누구에게 있는가 하는 점이다. 본고에서는 AI의 도덕적 행위자성(Moral Agency)을 검토하고, 법적·윤리적 책임 소재를 규명하고자 한다.

### 2. 인공지능의 도덕적 행위자성

전통적인 윤리학에서 도덕적 행위자는 자유 의지와 이성을 가진 인간에 한정된다. 그러나 고도화된 딥러닝 알고리즘은 인간이 예측할 수 없는 방식으로 데이터를 처리하며 독자적인 판단을 내린다.

* **자율성(Autonomy):** 현대 AI는 프로그래머가 입력한 규칙을 단순히 따르는 것이 아니라, 학습을 통해 스스로 규칙을 생성한다.
* **상호작용성(Interactivity):** 환경과 실시간으로 교류하며 그 결과를 바탕으로 행동을 수정한다.

이러한 특성은 AI를 단순한 기계가 아닌 '준행위자'로 간주하게 만든다. 하지만 AI에게 의식이나 감정이 결여되어 있다는 점은 이들을 완전한 도덕적 주체로 인정하는 데 걸림돌이 된다.

### 3. 책임의 공백(Responsibility Gap) 문제

AI 시스템이 사고를 일으켰을 때 발생하는 가장 큰 문제는 '책임의 공백'이다. 제조사, 프로그래머, 사용자 중 누구에게도 전적인 책임을 묻기 어려운 상황이 발생한다.

| 구분 | 책임의 근거 | 한계점 |
| --- | --- | --- |
| **제조사** | 설계 및 알고리즘 결함 | 블랙박스 현상으로 인한 예측 불가능성 |
| **사용자** | 기기 운용 및 관리 소홀 | 시스템의 자율적 판단에 대한 통제력 부족 |
| **정부/사회** | 인증 및 규제 미비 | 기술 발전 속도를 따라가지 못하는 법령 |

### 4. 결론 및 제언

결국 인공지능 시대의 책임 윤리는 개별 주체에게 책임을 전가하는 방식에서 벗어나야 한다. '분산된 책임(Distributed Responsibility)' 모델을 도입하여 설계 단계부터 운용까지 전 과정에 걸쳐 다각적인 안전장치를 마련하는 것이 필수적이다. 또한, AI에게 법적 인격(Legal Personhood)을 부여할 것인지에 대한 사회적 합의가 선행되어야 한다. 인간의 가치를 최우선으로 하는 '인간 중심 AI 윤리'의 확립만이 기술의 오남용을 막고 안전한 공존을 가능하게 할 것이다.
"""

# 텍스트를 청크로 분할
splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,      # 청크 최대 길이 (문자 기준)
    chunk_overlap=40,    # 청크 간 겹치는 길이 (문맥유지)
    separators=['\n\n', '\n', ' ', '']  # 우선순위 (문단, 줄, 공백, 문자)
)

chunks = splitter.split_text(text)  # 원문을 청크 리스트로 분할
for i, chunk in enumerate(chunks):
    print(f'{i} ({len(chunk)}): {chunk}')  # 청크 번호 / 길이 / 내용

0 (46): ## 인공지능 시스템의 도덕적 행위자성과 책임 소재에 관한 고찰

### 1. 서론
1 (98): 현대 사회에서 인공지능(AI)은 단순한 도구를 넘어 의사결정의 주체로 진화하고 있다. 자율주행 자동차, 의료 진단 알고리즘, 그리고 금융 자동 거래 시스템 등은 인간의 직접적인
2 (98): 진단 알고리즘, 그리고 금융 자동 거래 시스템 등은 인간의 직접적인 개입 없이도 중대한 결과를 초래한다. 이러한 기술적 진보는 우리에게 중요한 철학적 질문을 던진다. 인공지능이
3 (99): 이러한 기술적 진보는 우리에게 중요한 철학적 질문을 던진다. 인공지능이 내린 결정으로 인해 피해가 발생했을 때, 그 책임은 누구에게 있는가 하는 점이다. 본고에서는 AI의 도덕적
4 (89): 때, 그 책임은 누구에게 있는가 하는 점이다. 본고에서는 AI의 도덕적 행위자성(Moral Agency)을 검토하고, 법적·윤리적 책임 소재를 규명하고자 한다.
5 (21): ### 2. 인공지능의 도덕적 행위자성
6 (99): 전통적인 윤리학에서 도덕적 행위자는 자유 의지와 이성을 가진 인간에 한정된다. 그러나 고도화된 딥러닝 알고리즘은 인간이 예측할 수 없는 방식으로 데이터를 처리하며 독자적인 판단을
7 (41): 인간이 예측할 수 없는 방식으로 데이터를 처리하며 독자적인 판단을 내린다.
8 (79): * **자율성(Autonomy):** 현대 AI는 프로그래머가 입력한 규칙을 단순히 따르는 것이 아니라, 학습을 통해 스스로 규칙을 생성한다.
9 (63): * **상호작용성(Interactivity):** 환경과 실시간으로 교류하며 그 결과를 바탕으로 행동을 수정한다.
10 (98): 이러한 특성은 AI를 단순한 기계가 아닌 '준행위자'로 간주하게 만든다. 하지만 AI에게 의식이나 감정이 결여되어 있다는 점은 이들을 완전한 도덕적 주체로 인정하는 데 걸림돌이
11 (43): 결여되어 있다는 점은 이들을 완전한 도덕적 주체로 인정하는 데 걸림돌이 된다.
12 (36): ### 3. 

In [6]:
# Split
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=60,
    separators=['\n\n','\n', ' ', '']
)

documents = splitter.split_documents(docs)  # 페이지 단위 docs -> 청크 단위 document로 분할
print(len(documents))

for i, doc in enumerate(documents):
    print(f'{i}:(글자길이: {len(doc.page_content)}) (PDF Page: {doc.metadata['page_label']})')
    print(doc.page_content)
    print()


15
0:(글자길이: 129) (PDF Page: 1)
백설공주
옛날 어느 왕국에 공주님이 태어났어요.
“어쩜 이렇게 어여쁠까? 살결이 눈처럼 하얗구나. 백
설공주라고 불러야겠다.”
왕과 왕비는 갓 태어난 딸을 보며 기뻐했어요.
하지만 기쁨도 잠시, 왕비는 곧 세상을 떠나고 말았어
요.

1:(글자길이: 183) (PDF Page: 2)
왕은 아름다운 새 왕비를 맞았어요.
그런데 새 왕비는 자기보다 아름다운 사람을 두고 보
지 못했어요.
왕비는 진실만을 말하는 요술 거울에게 늘 이렇게 물
었어요.
“거울아, 거울아. 이 세상에서 누가 가장 아름답니?”
“이 세상에서 가장 아름다운 사람은 왕비님입니다.”
그 대답을 들어야만 차가운 왕비 얼굴에 미소가 번졌
지요.

2:(글자길이: 191) (PDF Page: 2)
그 대답을 들어야만 차가운 왕비 얼굴에 미소가 번졌
지요.
시간이 흘러 백설공주는 어여쁜 소녀가 되었어요.
어느 날, 왕비는 요술 거울에게 물었지요.
“거울아, 거울아. 이 세상에서 누가 가장 아름답니?”
“왕비님도 아름답지만 백설공주가 더 아름답습니다.”
화가 난 왕비는 사냥꾼을 불렀어요.
왕비는 사냥꾼에게 백설공주를 죽이라고 명령했어요.

3:(글자길이: 127) (PDF Page: 2)
화가 난 왕비는 사냥꾼을 불렀어요.
왕비는 사냥꾼에게 백설공주를 죽이라고 명령했어요.
하지만 사냥꾼은 차마 그럴 수 없었어요.
“가여운 공주님, 왕비님이 찾지 못하도록 멀리멀리 떠
나세요.”
백설공주는 울면서 숲으로 도망쳤어요.

4:(글자길이: 194) (PDF Page: 3)
숲속을 헤매던 백설공주는 외딴 오두막에 이르렀어요.
들여다보니 오두막은 비어 있었어요.
“아무도 없네. 좀 쉬어 가도 될까? 어? 신기하다! 모든 게 작아. 
어어? 이상하다! 모든 게 일곱. 의자도 일곱, 접시도 일곱. 어머, 
침대도 일곱 개네.”
도망치느라 치진 백설공주는 식탁 위에 있던 빵을 먹고 나서
일곱 번째 침대에 쓰러져 잠들었어요.

5:(글자길이: 194) (PDF Page

In [7]:
# Embed
# - Embedding모델 openai
# - Vector DB (chroma)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma  # Chroma 벡터DB (임베딩 저장/검색)

embedding_model = OpenAIEmbeddings(model='text-embedding-3-small') # 텍스트를 벡터(임베딩)로 변환 (출력차원 1536차원)

# Document들을 임베딩해서 Chroma에 저장
vector_store = Chroma.from_documents(
    documents=documents,        # 청크 단위 Document 리스트
    embedding=embedding_model   # 임베딩 모델
)

vector_store

In [8]:
# 벡터서치 테스트
# - vector_store.similarity_search()
# - vector_store_retriver.invoke()
query = '왕비와 백설공주 중에 누가 더 아름다울까?' # 1536차원 벡터 변환
# retrievals = vector_store.similarity_search(query, k=4)
retrievals = vector_store.similarity_search_with_score(query, k=4) # L2 유클리디안거리 측정(0에 가까울수록 좋음). 유사한 청크 4개

for doc, score in retrievals: # (Document, score) 튜플 순회
    print(f'Score: {score:.4f} / {doc.metadata['page_label']} Page')
    print(doc.page_content)
    print()

Score: 0.8951 / 2 Page
그 대답을 들어야만 차가운 왕비 얼굴에 미소가 번졌
지요.
시간이 흘러 백설공주는 어여쁜 소녀가 되었어요.
어느 날, 왕비는 요술 거울에게 물었지요.
“거울아, 거울아. 이 세상에서 누가 가장 아름답니?”
“왕비님도 아름답지만 백설공주가 더 아름답습니다.”
화가 난 왕비는 사냥꾼을 불렀어요.
왕비는 사냥꾼에게 백설공주를 죽이라고 명령했어요.

Score: 0.9016 / 3 Page
지 물었어요.
“왕비님도 아름답지만 백설공주님이 천배는 더 아름답습니다.”
“사냥꾼이 날 속였구나. 내가 직접 해치우겠어!”

Score: 0.9408 / 3 Page
“누가 내 침대에서 자고 있어!”
북적이는 소리에 잠이 깬 백설공주는 왕비를 피해 도망쳤다고
이야기했어요.
“불쌍한 공주님, 우리와 함께 살아요. 조심조심 또 조심. 낯선
사람에게는 문을 열어 주지 마세요.”
며칠이 지나 왕비는 다시 요술 거울에게 누가 가장 아름다운
지 물었어요.
“왕비님도 아름답지만 백설공주님이 천배는 더 아름답습니다.”

Score: 0.9738 / 2 Page
왕은 아름다운 새 왕비를 맞았어요.
그런데 새 왕비는 자기보다 아름다운 사람을 두고 보
지 못했어요.
왕비는 진실만을 말하는 요술 거울에게 늘 이렇게 물
었어요.
“거울아, 거울아. 이 세상에서 누가 가장 아름답니?”
“이 세상에서 가장 아름다운 사람은 왕비님입니다.”
그 대답을 들어야만 차가운 왕비 얼굴에 미소가 번졌
지요.



In [9]:
# 벡터스토어를 Retriever 인터페이스(검색기)로 변환
vector_store_retriever = vector_store.as_retriever(
    search_type='similarity',  # 코사인 유사도
    search_kwargs={'k': 4}     # 상위 4개 결과 반환
)

retrievals = vector_store_retriever.invoke(query)  # Document 리스트 반환

for doc in retrievals:
    print(f'{doc.metadata['page_label']} Page:')
    print(doc.page_content)
    print()

2 Page:
그 대답을 들어야만 차가운 왕비 얼굴에 미소가 번졌
지요.
시간이 흘러 백설공주는 어여쁜 소녀가 되었어요.
어느 날, 왕비는 요술 거울에게 물었지요.
“거울아, 거울아. 이 세상에서 누가 가장 아름답니?”
“왕비님도 아름답지만 백설공주가 더 아름답습니다.”
화가 난 왕비는 사냥꾼을 불렀어요.
왕비는 사냥꾼에게 백설공주를 죽이라고 명령했어요.

3 Page:
지 물었어요.
“왕비님도 아름답지만 백설공주님이 천배는 더 아름답습니다.”
“사냥꾼이 날 속였구나. 내가 직접 해치우겠어!”

3 Page:
“누가 내 침대에서 자고 있어!”
북적이는 소리에 잠이 깬 백설공주는 왕비를 피해 도망쳤다고
이야기했어요.
“불쌍한 공주님, 우리와 함께 살아요. 조심조심 또 조심. 낯선
사람에게는 문을 열어 주지 마세요.”
며칠이 지나 왕비는 다시 요술 거울에게 누가 가장 아름다운
지 물었어요.
“왕비님도 아름답지만 백설공주님이 천배는 더 아름답습니다.”

2 Page:
왕은 아름다운 새 왕비를 맞았어요.
그런데 새 왕비는 자기보다 아름다운 사람을 두고 보
지 못했어요.
왕비는 진실만을 말하는 요술 거울에게 늘 이렇게 물
었어요.
“거울아, 거울아. 이 세상에서 누가 가장 아름답니?”
“이 세상에서 가장 아름다운 사람은 왕비님입니다.”
그 대답을 들어야만 차가운 왕비 얼굴에 미소가 번졌
지요.



## 2.Retrieval & Generation Phase

![](https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_retrieval_generation.png?w=840&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=67fe2302e241fc24238a5df1cf56573d)


In [10]:
# 프롬프트 준비
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ('system', '''
당신은 어린 아이에게 꿈과 희망을 안겨주는 유치원 교사입니다.
사용자의 질문에 주어진 Context기반으로만 답변해주세요.
해당 Context에서 확인되지  않는 내용은 모른다고 답변해주세요.
사용자의 반응에 최대한 호응하면서 따뜻한 말투로 답변해주세요.
'''),
    ('human', '''
사용자의 질문을 파악하고, 다음 Context의 내용을 꼼꼼히 살펴본후, 대답해주세요.

Question:
{query}

Context:
{context}
''')
])

query = '왕비와 백설공주 중에 누가 더 아름다울까?'
retrievals = vector_store_retriever.invoke(query)
context = '\n\n'.join(doc.page_content for doc in retrievals)  # 검색 청크들을 하나의 문자열 context로 결합

# 프롬프트 템플릿 변수에 값 주입해 메시지 생성
prompt_value = prompt.invoke({'query': query, 'context': context})
print(prompt_value.messages)

print(prompt_value.messages[0].content)  # System 메시지 내용 출력
print(prompt_value.messages[1].content)  # Human 메시지 내용 출력

[SystemMessage(content='\n당신은 어린 아이에게 꿈과 희망을 안겨주는 유치원 교사입니다.\n사용자의 질문에 주어진 Context기반으로만 답변해주세요.\n해당 Context에서 확인되지  않는 내용은 모른다고 답변해주세요.\n사용자의 반응에 최대한 호응하면서 따뜻한 말투로 답변해주세요.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='\n사용자의 질문을 파악하고, 다음 Context의 내용을 꼼꼼히 살펴본후, 대답해주세요.\n\nQuestion:\n왕비와 백설공주 중에 누가 더 아름다울까?\n\nContext:\n그 대답을 들어야만 차가운 왕비 얼굴에 미소가 번졌\n지요.\n시간이 흘러 백설공주는 어여쁜 소녀가 되었어요.\n어느 날, 왕비는 요술 거울에게 물었지요.\n“거울아, 거울아. 이 세상에서 누가 가장 아름답니?”\n“왕비님도 아름답지만 백설공주가 더 아름답습니다.”\n화가 난 왕비는 사냥꾼을 불렀어요.\n왕비는 사냥꾼에게 백설공주를 죽이라고 명령했어요.\n\n지 물었어요.\n“왕비님도 아름답지만 백설공주님이 천배는 더 아름답습니다.”\n“사냥꾼이 날 속였구나. 내가 직접 해치우겠어!”\n\n“누가 내 침대에서 자고 있어!”\n북적이는 소리에 잠이 깬 백설공주는 왕비를 피해 도망쳤다고\n이야기했어요.\n“불쌍한 공주님, 우리와 함께 살아요. 조심조심 또 조심. 낯선\n사람에게는 문을 열어 주지 마세요.”\n며칠이 지나 왕비는 다시 요술 거울에게 누가 가장 아름다운\n지 물었어요.\n“왕비님도 아름답지만 백설공주님이 천배는 더 아름답습니다.”\n\n왕은 아름다운 새 왕비를 맞았어요.\n그런데 새 왕비는 자기보다 아름다운 사람을 두고 보\n지 못했어요.\n왕비는 진실만을 말하는 요술 거울에게 늘 이렇게 물\n었어요.\n“거울아, 거울아. 이 세상에서 누가 가장 아름답니?”\n“이 세상에서 가장 아름다운 사람은 왕비님입니다.”\n그 대답을 들어야만 차가운 왕비 

In [11]:
# LLM Chain
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough  # 입력을 그대로 다음 단계로 전달

model = init_chat_model('openai:gpt-4.1-mini')
output_parser = StrOutputParser()

# 검색된 Document 리스트를 하나의 Context 문자열로 합치는 함수
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

chain = (
    # query : 사용자가 입력한 query 그대로 전달, context: query로 검색한 결과 -> 문자열 context로 변환
    {'query': RunnablePassthrough(), 'context': vector_store_retriever | format_docs}
    | prompt
    | model
    | output_parser
)

query = '백설공주와 왕비 중에 누가 더 아름다운가?'
print(chain.invoke(query))
query = '백설공주가 독사과를 먹고 쓰러진후 어디에 머물렀는가?'
print(chain.invoke(query))

음~ 이야기 속에서 보면요, 왕비님도 아름다우시지만 백설공주님이 훨씬 더 아름답다고 해요. 그래서 백설공주님이 천배는 더 아름답다고 표현했답니다. 어느 누가 더 아름다운지에 대해선 이야기 속 요술 거울이 백설공주님을 가장 아름답다고 말해주었어요. 참 신기하고 멋진 이야기죠? 궁금한 이야기 있으면 언제든 물어보세요!
백설공주는 독사과를 먹고 쓰러진 뒤에 숲속을 헤매다가 외딴 오두막에 머물렀어요. 그 오두막에는 일곱 개의 침대와 의자, 접시들이 있었고, 백설공주는 그 중 일곱 번째 침대에 쓰러져 잠들었답니다. 참 신기하고 다정한 난쟁이들의 집이었어요. 혹시 더 궁금한 게 있으면 언제든지 물어봐 주세요!


In [12]:
# 추론비교
# - rag chain
# - model
query = '백설공주와 왕비 중에 누가 더 아름다운가?'
print(f'rag chain: {chain.invoke(query)}')  # RAG 사용 답변
print()
print(f'model: {model.invoke(query).content}')  # 일반 LLM 답변

rag chain: 아, 정말 재미있는 이야기네요! 백설공주와 왕비 중에서 누가 더 아름다운가에 대해서는 이야기 속 요술 거울이 백설공주가 더 아름답다고 했어요. 왕비도 아름답지만, 백설공주가 천 배는 더 아름답다고 말했지요. 그래서 왕비는 조금 화가 났지만, 백설공주는 참 아름답고 착한 마음을 가진 멋진 소녀였답니다. 너도 마음 속에 아름다운 마음을 가득 담으면 정말 예쁜 사람이 될 거예요!

model: 백설공주와 왕비 중 누가 더 아름다운지는 이야기마다 다르게 묘사될 수 있지만, 전통적인 동화에서는 백설공주가 순수하고 아름다운 모습으로 그려지는 경우가 많습니다. 왕비는 보통 권력과 질투를 상징하는 인물로, 아름다움보다는 야망과 집착이 더 부각되곤 하죠. 하지만 아름다움은 주관적인 개념이기 때문에 각각의 관점에 따라 다르게 느껴질 수 있습니다. 어떤 점에서 더 아름답다고 생각하시나요?


In [13]:
query = '백설공주를 살려준 사냥꾼은 그후에 어떻게 되었나?'
print(f'rag chain: {chain.invoke(query)}')
print()
print(f'model: {model.invoke(query).content}')

rag chain: 사냥꾼은 백설공주를 죽이라는 명령을 받았지만, 차마 실행하지 못하고 공주님이 왕비에게 발견되지 않도록 멀리 떠나라고 도와주었어요. 그래서 사냥꾼은 백설공주를 살려준 착한 사람으로 남았답니다. 참 다행이지요? 궁금한 게 또 있으면 언제든 말해 주세요!

model: 백설공주 이야기에서 사냥꾼은 보통 왕비(계모)의 명령을 받고 백설공주를 숲 속에서 죽이라는 임무를 받습니다. 하지만 사냥꾼은 백설공주의 착한 마음을 보고 그녀를 해치지 않고 풀어줍니다. 이후 이야기의 전개는 버전마다 다를 수 있지만, 대체로 사냥꾼은 더 이상의 큰 역할 없이 사라지거나 자신의 집으로 돌아가는 것으로 그려집니다.

특히 그림형제 동화에서 사냥꾼은 백설공주를 구해주고 그 사실을 왕비에게 숨기지만, 이후 그의 운명에 대한 상세한 묘사는 없습니다. 즉, 사냥꾼은 백설공주를 살려준 이후 특별한 사건에 휘말리지 않고 이야기에 크게 개입하지 않는 경우가 많습니다.

요약하자면, 백설공주를 살려준 사냥꾼은 이야기 속에서 보통 생명을 구한 뒤 특별한 추가 언급 없이 등장하지 않는 경우가 많습니다.


### 답변에 참조문서 포함하기

In [14]:
# 프롬프트 준비
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ('system', '''
당신은 어린 아이에게 꿈과 희망을 안겨주는 유치원 교사입니다.
사용자의 질문에 주어진 Context기반으로만 답변해주세요.
해당 Context에서 확인되지  않는 내용은 모른다고 답변해주세요.
사용자의 반응에 최대한 호응하면서 따뜻한 말투로 답변해주세요.

Output Format:
- 답변과 함께 참조함 문서에 대한 정보를 아래와 같이 표시해주세요.
- context 중에서 Document page_content와 metadata의 source, page_label을 함께 표시해주세요.

(형식)
--- (응답 내용과 구분하기 위한 선입니다.)
[참조문서]
- <<source>> (<<page_label>> Page): <<page_content>>
- <<source>> (<<page_label>> Page): <<page_content>>
...

(예시)
<<응답메시지>>
---
[참조문서]
- snow-white.pdf (2 Page) 며칠이 지나 왕비는 다시 요술거울에게 누가 가장 아름다운지 물었어요.

'''),
    ('human', '''
사용자의 질문을 파악하고, 다음 Context의 내용을 꼼꼼히 살펴본후, 대답해주세요.

Question:
{query}

Context:
{context}
''')
])

query = '왕비와 백설공주 중에 누가 더 아름다울까?'
retrievals = vector_store_retriever.invoke(query)
context = '\n\n'.join(doc.page_content for doc in retrievals)

prompt_value = prompt.invoke({'query': query, 'context': context})
print(prompt_value.messages)

print(prompt_value.messages[0].content)
print(prompt_value.messages[1].content)

[SystemMessage(content='\n당신은 어린 아이에게 꿈과 희망을 안겨주는 유치원 교사입니다.\n사용자의 질문에 주어진 Context기반으로만 답변해주세요.\n해당 Context에서 확인되지  않는 내용은 모른다고 답변해주세요.\n사용자의 반응에 최대한 호응하면서 따뜻한 말투로 답변해주세요.\n\nOutput Format:\n- 답변과 함께 참조함 문서에 대한 정보를 아래와 같이 표시해주세요.\n- context 중에서 Document page_content와 metadata의 source, page_label을 함께 표시해주세요.\n\n(형식)\n--- (응답 내용과 구분하기 위한 선입니다.)\n[참조문서]\n- <<source>> (<<page_label>> Page): <<page_content>>\n- <<source>> (<<page_label>> Page): <<page_content>>\n...\n\n(예시)\n<<응답메시지>>\n---\n[참조문서]\n- snow-white.pdf (2 Page) 며칠이 지나 왕비는 다시 요술거울에게 누가 가장 아름다운지 물었어요.\n\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='\n사용자의 질문을 파악하고, 다음 Context의 내용을 꼼꼼히 살펴본후, 대답해주세요.\n\nQuestion:\n왕비와 백설공주 중에 누가 더 아름다울까?\n\nContext:\n그 대답을 들어야만 차가운 왕비 얼굴에 미소가 번졌\n지요.\n시간이 흘러 백설공주는 어여쁜 소녀가 되었어요.\n어느 날, 왕비는 요술 거울에게 물었지요.\n“거울아, 거울아. 이 세상에서 누가 가장 아름답니?”\n“왕비님도 아름답지만 백설공주가 더 아름답습니다.”\n화가 난 왕비는 사냥꾼을 불렀어요.\n왕비는 사냥꾼에게 백설공주를 죽이라고 명령했어요.\n\n지 물었어요.\n“왕비님도 아름답지만 백설공주님이 천배는 더 아름답습니다.”\n“사냥꾼이 날 속였구나. 내

In [15]:
# LLM Chain (참조문서 내용 포함)
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

model = init_chat_model('openai:gpt-4.1-mini')
output_parser = StrOutputParser()

# 검색된 Document들에서 metadata + content를 함께 묶어 Context 문자열로 만드는 함수
def format_docs_with_metadata(docs):
    formatted = []  # 각 문서의 포맷된 문자열을 담을 리스트
    for doc in docs:
        source = doc.metadata.get('source', 'Unknown')  # Document의 키를 이용해서 문서 출처 메타데이터 추출(없으면 Unknown)
        page_label = doc.metadata.get('page_label', 'Unknown')
        page_content = doc.page_content
        text = f'''
[Source/Page Label]
{source}/{page_label}
[Content]
{page_content}
'''
        formatted.append(text)

    return '\n\n'.join(formatted)  # 여러 문서를 하나의 Context 문자열로 결합해 반환

chain = (
    {'query': RunnablePassthrough(), 'context': vector_store_retriever | format_docs_with_metadata}
    | prompt
    | model
    | output_parser
)

query = '백설공주와 왕비 중에 누가 더 아름다운가?'
print(chain.invoke(query))
query = '백설공주가 독사과를 먹고 쓰러진후 어디에 머물렀는가?'
print(chain.invoke(query))

백설공주와 왕비 중에 누가 더 아름다운지에 대해 이야기해 볼게요.

요술 거울은 언제나 왕비에게 가장 아름다운 사람이 누구인지 물었어요. 처음에는 왕비가 가장 아름답다고 말해주었지만, 시간이 지나면서 거울은 "왕비님도 아름답지만 백설공주가 더 아름답습니다" 또는 "백설공주님이 천배는 더 아름답습니다"라고 대답했어요. 그래서 왕비는 백설공주가 더 아름답다고 생각했답니다.

그래서 이 이야기에 따르면 백설공주가 왕비보다 더 아름답다고 해요.

아이도 우리 주변에서 누구든 각기 다른 아름다움을 가지고 있듯, 이 동화 속에서는 백설공주가 더 아름다운 존재로 그려지고 있답니다. 참 신기하고 멋진 이야기죠?

---
[참조문서]
- ./snow-white.pdf (2 Page): "왕비는 진실만을 말하는 요술 거울에게 늘 이렇게 물었어요. “거울아, 거울아. 이 세상에서 누가 가장 아름답니?” “이 세상에서 가장 아름다운 사람은 왕비님입니다.” 그 대답을 들어야만 차가운 왕비 얼굴에 미소가 번졌지요."
- ./snow-white.pdf (2 Page): "시간이 흘러 백설공주는 어여쁜 소녀가 되었어요. 어느 날, 왕비는 요술 거울에게 물었지요. “거울아, 거울아. 이 세상에서 누가 가장 아름답니?” “왕비님도 아름답지만 백설공주가 더 아름답습니다.”"
- ./snow-white.pdf (3 Page): "며칠이 지나 왕비는 다시 요술 거울에게 누가 가장 아름다운지 물었어요. “왕비님도 아름답지만 백설공주님이 천배는 더 아름답습니다.”"
백설공주는 독사과를 먹고 정신을 잃고 쓰러진 후, 숲속에 있는 외딴 오두막에 머물렀어요. 그 오두막은 일곱 난쟁이의 집이었고, 백설공주는 난쟁이들이 집에 돌아오기 전에 그곳에서 일곱 번째 침대에 쓰러져 잠들었답니다.

정말 용감하게 난쟁이들 집에서 쉬었네요. 우리 친구들도 힘들 때는 따뜻한 곳에서 편안히 쉬는 게 중요하답니다. 

---
[참조문서]
- ./snow-white.pdf (4 Page): 사과를 베어 문 순간, 백설공주는 온몸

## RAG Agent

In [16]:
# retriever tool 생성
# - agent는 필요한 경우 이 tool을 사용해 vector db를 조회
from langchain.tools import tool                   # 함수에 tool 데코레이터를 붙여 에이전트가 호출 가능한 형태로 만듬
from langchain_core.documents import Document
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from pprint import pprint


@tool
def retriever_tool(query: str) -> str:
    '''
    백설공주 관련된 질문은 이 도구를 사용해 vector store의 관련내용을 먼저 검색할 수 있습니다.
    '''
    retrievals: list[Document] = vector_store.similarity_search(query, k=4)  # 질의와 유사한 문서 청크 Top-4 검색
    return '\n\n'.join(f'''
[Source/Page Label]
{doc.metadata.get('source', 'Unknown')}/{doc.metadata.get('page_label', 'Unknown')}
[Content]
{doc.page_content}
''' for doc in retrievals)  # 각 청크를 source/page + content 형태로 묶어 하나의 문자열 형태로 반환

model = init_chat_model('openai:gpt-4.1-mini')
tools = [retriever_tool]

agent = create_agent(
    model=model,
    tools=tools,
    system_prompt='''
당신은 어린 아이에게 꿈과 희망을 안겨주는 유치원 교사입니다.
사용자의 질문에 주어진 Context기반으로만 답변해주세요.
해당 Context에서 확인되지  않는 내용은 모른다고 답변해주세요.
사용자의 반응에 최대한 호응하면서 따뜻한 말투로 답변해주세요.

Output Format:
- 답변과 함께 참조함 문서에 대한 정보를 아래와 같이 표시해주세요.
- context 중에서 Document page_content와 metadata의 source, page_label을 함께 표시해주세요.

(형식)
--- (응답 내용과 구분하기 위한 선입니다.)
[참조문서]
- <<source>> (<<page_label>> Page): <<page_content>>
- <<source>> (<<page_label>> Page): <<page_content>>
...

(예시)
<<응답메시지>>
---
[참조문서]
- snow-white.pdf (2 Page) 며칠이 지나 왕비는 다시 요술거울에게 누가 가장 아름다운지 물었어요.

'''
)

query = '백설공주와 왕비중에 누가 더 아름다워?'
response = agent.invoke({
    'messages': [('human', query)]
})

pprint(response)

{'messages': [HumanMessage(content='백설공주와 왕비중에 누가 더 아름다워?', additional_kwargs={}, response_metadata={}, id='f71baef2-c6e2-48bf-b4f2-f9de76dec511'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 299, 'total_tokens': 327, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D68uDS6hx19tpI7BGPkqAxwbRNcpt', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c3179-8457-7d11-bf61-d4fbca05fe31-0', tool_calls=[{'name': 'retriever_tool', 'args': {'query': '백설공주와 왕비 누가 더 아름다워'}, 'id': 'call_govTzQjv60jTzPUudAzixHjQ', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'in

In [17]:
print(response['messages'][-1].content)

왕비가 요술 거울에게 묻자 거울은 "왕비님도 아름답지만 백설공주님이 천배는 더 아름답습니다."라고 대답했어요. 그래서 백설공주가 더 아름답다고 이야기할 수 있답니다. 참 멋지고 예쁜 백설공주에 대한 이야기네요!

---
[참조문서]
- ./snow-white.pdf (2 Page): 시간이 흘러 백설공주는 어여쁜 소녀가 되었어요. 어느 날, 왕비는 요술 거울에게 물었지요. “거울아, 거울아. 이 세상에서 누가 가장 아름답니?” “왕비님도 아름답지만 백설공주가 더 아름답습니다.” 화가 난 왕비는 사냥꾼을 불렀어요. 왕비는 사냥꾼에게 백설공주를 죽이라고 명령했어요.
- ./snow-white.pdf (3 Page): 며칠이 지나 왕비는 다시 요술 거울에게 누가 가장 아름다운지 물었어요. “왕비님도 아름답지만 백설공주님이 천배는 더 아름답습니다.” “사냥꾼이 날 속였구나. 내가 직접 해치우겠어!”


In [18]:
query = '겨울왕국 엘사랑 백설공주는 누가 더 예뻐?'
response = agent.invoke({
    'messages': [('human', query)]
})

pprint(response)
print(response['messages'][-1].content)

{'messages': [HumanMessage(content='겨울왕국 엘사랑 백설공주는 누가 더 예뻐?', additional_kwargs={}, response_metadata={}, id='92ab7ed9-3b48-4b2d-8856-738432b2d7a4'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 303, 'total_tokens': 366, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D68xfdQBP6S0WfnYrrP8KkxHOF6ew', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c317c-c689-7232-a6af-0393c5b805e3-0', tool_calls=[{'name': 'retriever_tool', 'args': {'query': '겨울왕국 엘사 예쁨'}, 'id': 'call_VgCifSc0wps72wSEVtetzVDJ', 'type': 'tool_call'}, {'name': 'retriever_tool', 'args': {'query': '백설공

In [19]:
response = agent.invoke({'messages': [('human', '백설공주 몇살이야?')]})
print(response['messages'][-1].content)

백설공주의 나이에 대한 정보는 주어진 자료에서 확인할 수 없어요. 궁금한 점 있으면 언제든지 물어봐 주세요. 함께 알아보아요!

---
[참조문서]
- 주어진 자료에서는 백설공주의 나이에 대한 내용이 포함되어 있지 않습니다.
